In [3]:
from selenium import webdriver
import pandas as pd
import os
import datetime
from multiprocessing.pool import ThreadPool
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.options import Options
import glob
from bs4 import BeautifulSoup
import shutil
chromeOptions = Options()
from time import sleep
import datetime
import json
import re
import numpy as np

In [32]:
def get_stock_info(start, stop):
        
        driver = webdriver.Chrome(
        ChromeDriverManager().install(),
                )
        today = str(datetime.date.today())

        df = pd.read_csv("data/grow/{}/stocks.csv".format(today))
        urls = df["href"].iloc[start:stop].tolist()
        names = df["name"].iloc[start:stop].tolist()
        stock = {}
        for idx, (name, url) in enumerate(zip(names, urls)):
                root_dir = "data/stock"
                path = os.path.join(root_dir, "{}.json".format(idx + start))
                driver.get(url)
                try:
                        element = driver.find_element(By.XPATH, "//body[1]/div[1]/div[2]/div[2]/div[1]/div[1]/div[1]/div[1]/div[1]/div[1]/div[2]/div[1]/div[1]/div[4]/div[1]/section[1]/section[1]/div[1]/div[1]/div[2]/div[1]/div[2]/div[1]")
                        stock["todaymin"] = element.get_attribute("aria-valuemin")
                        stock["todaymax"] =  element.get_attribute("aria-valuemax")
                        stock["todaynow"] = element.get_attribute("aria-valuenow")
                        element = driver.find_element(By.XPATH, "//body[1]/div[1]/div[2]/div[2]/div[1]/div[1]/div[1]/div[1]/div[1]/div[1]/div[2]/div[1]/div[1]/div[4]/div[1]/section[1]/section[1]/div[1]/div[1]/div[2]/div[2]/div[2]/div[1]")
                        stock["52min"] = element.get_attribute("aria-valuemin")
                        stock["52max"] =  element.get_attribute("aria-valuemax")
                        stock["52now"] = element.get_attribute("aria-valuenow")
                        xpath = "//body/div[@id='__next']/div[@id='root']/div/div[contains(@class,'container web-align')]/div[@class='row pw14Container']/div[@class='pw14MainWrapper']/div[@class='pw14ContentWrapper']/div[@class='col l12']/div[@class='container web-align']/div[@class='col l12 stkP12BelowFoldDiv']/div[@class='col l12 stkP12BelowFoldInnerCont']/div[@class='col l12']/div[@class='col l12']/div[@class='col l12']/section/section/div[@class='onMount-appear-done onMount-enter-done']/div[1]"
                        change_price = driver.find_element(By.XPATH, "//div[@class='lpu38Day fs14 primaryClr']")
                        stock["price_change"] = change_price.text
                        div = driver.find_element(By.XPATH, xpath)
                        innerhtml = div.get_attribute("innerHTML")
                        soup = BeautifulSoup(innerhtml, "html.parser")
                        names = [i.text for i in soup.find_all("div", {"class" : "stpp34KeyText stpp34KeyTextStk fs14"})]
                        values = [i.text for i in soup.find_all("span", {"class": "stpp34Value fs16"})]
                        element = driver.find_element(By.XPATH, "//table[@class='tb10Table col l12 ft785Table']")
                        trs = element.find_elements(By.TAG_NAME, "tr")
                        for i in range(len(trs)):
                                tds = trs[i].find_elements(By.TAG_NAME, "td")
                                names.append(tds[0].text)
                                values.append(tds[1].text)
                        xpath =  "//div[@class='col l12 stkP12Section onMount-appear-done onMount-enter-done']//table[1]"
                        element = driver.find_element(By.XPATH, xpath)
                        trs = element.find_elements(By.TAG_NAME, "tr")
                        for i in range(len(trs)):
                                tds = trs[i].find_elements(By.TAG_NAME, "td")
                                names.append(tds[0].text)
                                values.append(tds[1].text)
                        xpath = "//div[@class='col l12 stkP12Section onMount-appear-done onMount-enter-done']//table[2]"
                        element = driver.find_element(By.XPATH, xpath)
                        trs = element.find_elements(By.TAG_NAME, "tr")
                        for i in range(len(trs)):
                                tds = trs[i].find_elements(By.TAG_NAME, "td")
                                if "Symbol" in tds[0].text:
                                        names.append("Symbol")
                                else:
                                        names.append(tds[0].text)
                                values.append(tds[1].text)
                        stock.update(dict(zip(names, values)))
                        
                        
                        stock["name"] = name
                except:
                        continue
                # print(path)
                print(path)
                with open(path, 'w') as f:
                        json.dump(stock, f, indent=4)
        driver.close()

In [33]:
get_stock_info(0, 10)

data/stock/0.json
data/stock/2.json
data/stock/3.json
data/stock/4.json
data/stock/5.json
data/stock/6.json
data/stock/8.json
data/stock/9.json


In [14]:
url = "https://groww.in/stocks/reliance-industries-ltd"
driver = webdriver.Chrome(
        ChromeDriverManager().install(),
                )
driver.get(url)

In [5]:
curr_price = driver.find_element(By.XPATH, "//div[@class='lpu38Pri valign-wrapper false fs28']")

In [7]:
change_price = driver.find_element(By.XPATH, "//div[@class='lpu38Day fs14 primaryClr']")

In [8]:
change_price.text

'+43.45 (1.83%) 1D'

In [18]:
todays_low = driver.find_element(By.XPATH, "//body[1]/div[1]/div[2]/div[2]/div[1]/div[1]/div[1]/div[1]/div[1]/div[1]/div[2]/div[1]/div[1]/div[4]/div[1]/section[1]/section[1]/div[1]/div[1]/div[2]/div[1]/div[2]/div[1]")